In [1]:
import pandas as pd
import numpy as np

#Removing some columns
cr = pd.read_csv("..//data//crimes.csv")

cr = cr.drop(['ID_delegacia','Nome_departamento','Nome_seccional','NOME_DELEGACIA','Cidade','Nome_departamento_circ','Nome_seccional_circ','Nome_delegacia_circ','Município','Tipo_BO','Apresentação','Autoria_BO','Intolerância','Rubrica','Desdobramento','Subtipo_local'], axis=1)

cr['LATITUDE'] = cr['LATITUDE'].fillna(0)
cr['LONGITUDE'] = cr['LONGITUDE'].fillna(0)
cr['Num_logradouro'] = cr['Num_logradouro'].fillna(0)

#Removing some rows in which it is impossible find out lat and lon
for index, row in cr.iterrows():
    if (row['LATITUDE'] == 0):
        if (pd.isnull(row['CEP']) or row['CEP'] == '-'):
            if (pd.isnull(row['LOGRADOURO']) or row['Num_logradouro']==0):
                cr = cr.drop(index)    

#getting some missing crimes
cr2 = pd.read_csv("..//data//crimes.csv")
cr2 = cr2.drop(['ID_delegacia','Nome_departamento','Nome_seccional','NOME_DELEGACIA','Cidade','Nome_departamento_circ','Nome_seccional_circ','Nome_delegacia_circ','Município','Tipo_BO','Apresentação','Autoria_BO','Intolerância','Rubrica','Desdobramento','Subtipo_local'], axis=1)
cr2['LATITUDE'] = cr2['LATITUDE'].fillna(0)
cr2['LONGITUDE'] = cr2['LONGITUDE'].fillna(0)
cr2['Num_logradouro'] = cr2['Num_logradouro'].fillna(0)

for index, row in cr2.iterrows():
    if (row['LATITUDE'] == 0):
        if (pd.isnull(row['LOGRADOURO']) or row['Num_logradouro']==0):
            cr2 = cr2.drop(index)
            
new = pd.concat([cr, cr2]).drop_duplicates(keep=False)

new.to_csv("..//data//missing_crimes.csv",index=None)

In [72]:
#joining crimes

In [8]:
from datetime import datetime
import random

cr = pd.read_csv("..//data//crimes_final.csv")


cr = cr[cr['Conduta'].isin(['ESTABELECIMENTO COMERCIAL','INTERIOR ESTABELECIMENTO','ESTABELECIMENTO ENSINO','ESTABELECIMENTO-OUTROS','TRANSEUNTE','VEICULO'])]

def class_cr(cr):
    if (cr['Conduta'] == 'ESTABELECIMENTO COMERCIAL' or cr['Conduta'] == 'INTERIOR ESTABELECIMENTO' or cr['Conduta'] == 'ESTABELECIMENTO ENSINO' or cr['Conduta'] == 'ESTABELECIMENTO-OUTROS'):
        return 'ESTABELECIMENTO'
    elif (cr['Conduta'] == 'TRANSEUNTE'):
        return 'TRANSEUNTE'
    elif (cr['Conduta'] == 'VEICULO'):
        return 'VEICULO'
    

cr['class'] = cr.apply(class_cr, axis = 1)

def generateHour(start,end):
    hours = random.randint(start, end)
    minutes = random.randint(0, 59)
    seconds = random.randint(0, 59)
    composto = datetime.strptime(str(hours)+':'+str(minutes)+':'+str(seconds),'%H:%M:%S')
    return composto.time()

def inputHour(data):
    for index, row in data.iterrows():
        h = ''
        if (pd.isnull(row['Hora_ocorrencia_BO'])):
            if (row['Período'] == 'DE MADRUGADA'):
                h = generateHour(0,5)
            elif (row['Período'] == 'PELA MANHA'):
                h = generateHour(6,11)
            elif (row['Período'] == 'A TARDE'):
                h = generateHour(12,17)
            elif (row['Período'] == 'A NOITE'):
                h = generateHour(18,23)
            else:  
                h = generateHour(0,23)
            data.loc[index,'Hora_ocorrencia_BO'] = h
    return data

cr = inputHour(cr)

cr.to_csv("..//data//crimes_final.csv",index=None)
